NOTRE KERNEL :

1. Compréhension du problème lié au Business

Toutes ces données : pourquoi faire ?

Exemple :

Une banque souhaite pouvoir automatiser la lecture des montants sur les chèques.
Pour cela, nous sommes sollicités afin de fournir un programme capable de lire des chiffres écrits à la main

2. Compréhension des données : Analyse des données d'entraînement

Dans cette partie, on réalise les opérations à la main pour comprendre les données :

a) Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.io
import warnings

from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

b) Chargement des données :

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

combine = [train_df, test_df]

c) Analyse des données :

On récupère dans un premier temps le pattern des données, présent dans le document d'entraînement "train.csv" :

In [0]:
print(train_df.columns.values)

Cette commande nous indique qu'il y a en tout 785 colonnes :

- La 1ère colonne correspond à la colonne des étiquettes
- Les autres colonnes correspondent à la suite des valeurs des pixels de chaque image

Ainsi, chaque ligne du fichier, à l'exception de la 1ère qui correspond au pattern présenté ci-dessus, représente une image par la suite des valeurs de ses pixels. Les images font donc 784 pixels chacune.

Le 1er élément de la ligne est l'étiquette, ici c'est un nombre qui correspond au digit (nombre) que représente l'image.
Nous allons donc réaliser du Machine Learning supervisé.

Visualisation et description générale des données :

In [0]:
# Prévisualisation des données
print(train_df.head())

# Description des données
print(train_df.info())

On peut voir que l'objet pandas qui a servi au chargement des données, retourne un objet de type Data Frame.
Cette commande nous indique également qu'il y a 42000 lignes :

- Comme dit précédemment, la 1ère est le pattern
- Les 41999 autres lignes sont les images

Les valeurs stockées sont toutes des entiers.

Les 785 colonnes contiennent toutes des valeurs catégoriques (à l'exception de la première ligne qui contient les noms des colonnes).

Les données présentent-elles des trous ou des valeurs non entières ?

In [0]:
def containsOnlyIntegerValues(matrix):

	for line in range(len(matrix)):
		for col in range(len(matrix[0])):
			try :
				int(matrix[line][col])
			except :
				return False

	return True

matrix = train_df.values.tolist()
print(containsOnlyIntegerValues(matrix))

Cette fonction nous apprend que nos données sont correctement formées et toutes la valeurs sont bien des entiers.
Il n'y a pas de valeur manquante.

Étudions maintenant la plage de données de chaque colonne :

In [0]:
def getColumnsRange(matrix):

	output = []

	for col in range(len(matrix[0])):
		colRange = []
		for line in range(len(matrix)):
			value = matrix[line][col]
			if not (value in colRange):
				colRange.append(value)
			colRange.sort()

		output.append(colRange)

	return output

print(train_df.describe())
print(getColumnsRange(matrix))

On observe ainsi que la 1ère colonne, la colonne des étiquette prend les valeurs suivantes : de 0 à 9

Et les autres colonnes prennent les valeurs suivantes :  de 0 à 255
Cela confirme donc que les données contiennent des images représentant les chiffres 0, 1, 2, ..., 9
En d'autre terme une image représente bien un unique chiffre et ce chiffre peut être un 0, un 1, ... ou un 9.
De plus, les images sont représentées en niveaux de gris et sont codées sur 8 bits.

On peut donc visualiser la 2e image des données :

Étudions maintenant la distribution des données :

In [0]:
def getDateDistribution(matrix):
	
	dict = {}
	numberOfLines = len(matrix)

	for line in range(numberOfLines):
		value = matrix[line][0]

		if value in dict:
			occurrences = dict[value]
			occurrences += 1
			dict[value] = occurrences

		else:
			dict[value] = 1

	for value in dict:
		occurrences = dict[value]
		percentage = (occurrences * 100) / numberOfLines
		dict[value] = percentage

	return dict

dict = getDateDistribution(matrix)

for i in dict:
	print("Digit : " + str(i) + "\t" + str(dict[i]) + "%")

Digit 0 : 9.838095238095239%

Digit 1 : 11.152380952380952%

Digit 2 : 9.945238095238095%

Digit 3 : 10.35952380952381%

Digit 4 : 9.695238095238095%

Digit 5 : 9.035714285714286%

Digit 6 : 9.85%

Digit 7 : 10.478571428571428%

Digit 8 : 9.673809523809524%

Digit 9 : 9.971428571428572%

On observe ainsi que les données sont réparties dans les bonnes proportions : il y a quasiment autant de 0 que de 1 que de 2, etc ...

Mais les images sont-elles réparties de manière uniforme ?
Pour le savoir, dressons l'histogramme :

b) Analyse des données :

L'analyse doit permettre de répondre aux questions suivantes :

Code TO DO

What is the meaning of each feature?
Which features are mixed data types?
Which features may contain errors or typos?
Which features contain blank, null or empty values?
What are the data types for various features?
What is the distribution of numerical feature values across the samples?
The training dataset is balanced or not? What is the main drawback of an unbalanced training dataset?
What is the distribution of categorical features?

c) Hypothèses basées sur l'analyse des données

On cherche les colonnes qu'on puisse supprimer, ajouter pour gagner du temps de calcul / gagner en qualité de prédiction ?
Pour cela, on cherche les corrélations, etc ...

Utilisation de PCA ?
Normalisation ?

3. Préparation des données

Code TO DO

On met en oeuvre nos hypothèses
À la fin de cette partie, nos données doivent être prêtes et dans le bon format pour les passer au modèle de ML choisi (ex : Logistic regression)

4. Modèle

On choisit un modèle de ML (exemple : Logistic Regression)
On justifie pourquoi ce modèle est bien pour notre problème.
On regarde quels sont les paramètres les plus adaptés à lui passer.

In [0]:
Code TO DO

5. Entraînement et évaluation du modèle

Découpage des données.
Il y a énormément de lignes, on prend tout ?
Validation croisée ?
Les données sont-elles mélangées uniformément (censé être déterminé dans l'analyse).

Code TO DO

6. Comparaison de nos résultats avec les résultats qu'on aurait eu sans appliquer nos hypothèses

Code TO DO

7. Prédictions

À ce stade notre code est prêt. On l'applique aux données de test (on a pas les réponses, donc cette partie sert juste à dire qu'il faut qu'on le fasse comme dans le cadre d'un projet d'entreprise)

Code TO DO

7. Sauvegarde du modèle dans un fichier

Pareil, c'est pour faire les mecs qui pèsent

Code TO DO

NOTEBOOK KERNEL QU'ON SUIT :

## Intoduction
link to competition: [digit recogniger competition](https://www.kaggle.com/c/digit-recognizer)

My Kaggle Profile: [Amit Vikram | Kaggle](https://www.kaggle.com/amitkvikram)

- It gives me immense pleasure to launch this kernel. For your sake of convenience I got accuracy of **0.98442**.

- Here we will use PCA for dimensionality reduction and then train the data using "Logistic Regression with solver lbfgs" and  "SVM". 

## 1. Import data

In [0]:
"""
Plan :

1) On s'inspire du titanic (on ne coule pas) :
   on analyse les données à la main pour regarder à chaque colonne :
   la moyenne, le nombre de valeurs différentes 
"""

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.io
import warnings

Train = pd.read_csv("train.csv").values
Test = pd.read_csv("test.csv").values

In [3]:
Train.shape

NameError: name 'Train' is not defined

In [4]:
Test.shape

NameError: name 'Test' is not defined

In [5]:
X = (Train[:,1:])
Y = (Train[:,0])

NameError: name 'Train' is not defined

## 2. Dimensionality Reduction using PCA

#### a. Plot graph of component vs. cumulative explained variance .
This graph will help us in choosing the no of components for training our data.

In [0]:
from sklearn.decomposition import PCA
# ## Removing the column with variance 0
variance = np.var(X, axis = 0)>1000
print(variance.shape)
X = X[:, variance]
Test = Test[:,variance]
print(X.shape)
# ##Calculate Principal Components
pca = PCA()
pca.fit(X)
explained_variance = pca.explained_variance_ratio_
# ##Calculate cumulative explained ration
cum_explained_variance = [np.sum(explained_variance[:i+1]) for i in range (0, 201, 25)]
X_axis = [i for i in range(0, 201,25)]

##Plot Graph
fig = plt.figure(figsize = (5.841, 7.195), dpi=100)
plt.plot(X_axis, cum_explained_variance, 'ro')
plt.grid(True, which = 'both')
plt.yticks(cum_explained_variance)
plt.xticks(X_axis)
plt.ylabel("Explained Variance Ratio")
plt.xlabel("No. of Components")
plt.show()

**So Looking on the above graph, 50 components comprise 80% variance. So first we will go with 50 componets**

### Splitting data for training and testing
- Training data: 80%
- Test data: 20%

In [1]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size  = 0.20, random_state  = 0)

NameError: name 'X' is not defined

### define normalize function for normalizing the data, PrincipalComponents function to return top n principal components.

In [0]:
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler

def normalize(sigma2, mean2, X):
    X = (X-mean2)/sigma2
    return X

def PolynomialFeatures1(X):
    X_2 = np.square(X)
    X = np.column_stack((X, X_2))
    return X

J1 = []
J2 = []

##Take n principal components
def PrincipalComponents(n):
    pca = PCA(n_components= n)
    X_train1 = pca.fit_transform(X_train)
    X_test1 = pca.transform(X_test)
    return X_train1, X_test1

## Logistic Regression

def LogisticRegression(X_train2, y_train2, X_test2, y_test2, penalty):
    print("penalty= ", penalty)
    regr = linear_model.LogisticRegression(solver='lbfgs',max_iter=1000, C=penalty)
    regr.fit(X_train2, y_train2)
    score1 = regr.score(X_train2, y_train2)
    score2 = regr.score(X_test2, y_test2)
    print(score1, score2)
    Prediction = regr.predict(X_test2)
    return score1, score2, Prediction


**So usually variance of 0.8 is sufficient to explain the variation in data, so we will first train data by taking the top n principal components which can explaine the variance of 0.8**

In [0]:
X_train1, X_test1 = PrincipalComponents(0.8)   # Getting principal components
J1 = []
J2 = []
for i in range(20000, X_train1.shape[0], 1500):
    score1, score2, Prediction = LogisticRegression(X_train1[:i+1,:], y_train[:i+1, ], 
                                                    X_test1[:i+1,:], y_test[:i+1], 0.1)
    J1.append(1-score1)
    J2.append(1-score2)
    
    
plt.plot(J2, 'b-', label = "CV error")
plt.plot(J1, 'r-', label ="training error")
plt.legend()
plt.ylabel("Error")
plt.xlabel("No of Iterations")
plt.show()

- We got a very irregular graph and not a satisfied accuracy and the accuracy doesn't increase with the increaing training data. So it seems that we are suffering from a high bias. Let us try to train data with top n components explaining the variance of 0.9

In [0]:
X_train1, X_test1 = PrincipalComponents(0.9)
J1 = []
J2 = []
for i in range(20000, X_train1.shape[0], 1500):
    score1, score2, Prediction = LogisticRegression(X_train1[:i+1,:],
                                                    y_train[:i+1, ], X_test1[:i+1,:], y_test[:i+1],0.1)
    J1.append(1-score1)
    J2.append(1-score2)
    
    
plt.plot(J2, 'b-', label = "CV error")
plt.plot(J1, 'r-', label ="training error")
plt.legend()
plt.ylabel("Error")
plt.xlabel("No of Iterations")
plt.show()

** We can see that accuracy didn't increase much so it doesn't seems a gud idea. Let us again take top n components explaining the variance of 0.8 and also include thieir polynomials(degree = 2)**.

#### Add polynomial features

##### 1.Logistic Regression

In [0]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(include_bias = False)
X_train1, X_test1 = PrincipalComponents(0.81)
print(X_test1.shape, X_train1.shape)

X_train1 = poly.fit_transform(X_train1)
X_test1 = poly.transform(X_test1)

print(X_test1.shape, X_train1.shape)

sigma = np.std(X_train1, axis = 0)
mean = np.mean(X_train1, axis = 0)

X_train1 = normalize(sigma , mean , X_train1)
X_test1 = normalize(sigma , mean, X_test1)

J1 = []
J2 = []
for i in range(25000, X_train1.shape[0], 1500):
    score1, score2, Prediction = LogisticRegression(X_train1[:i+1,:], 
                                                    y_train[:i+1, ], X_test1[:i+1,:], y_test[:i+1],0.1)
    J1.append(1-score1)
    J2.append(1-score2)
    
    
plt.plot(J2, 'b-', label = "CV error")
plt.plot(J1, 'r-', label ="training error")
plt.legend()
plt.ylabel("Error")
plt.xlabel("No of Iterations")
plt.show()

** We got way better accuracy here and is data over  fitting .... may be but since we are getting good result on cross validation data also, It's gud to go with this.**

#### Saving data trained with logistic regression

In [0]:
from sklearn.preprocessing import PolynomialFeatures
# poly = PolynomialFeatures(include_bias = False)

X1 = X.copy()
Test1 = Test.copy()

pca = PCA(n_components= 0.8388)
X1 = pca.fit_transform(X1)
Test1 = pca.transform(Test)

print(X1.shape, Test1.shape)

X1 = poly.fit_transform(X1)
Test1 = poly.fit_transform(Test1)

print(X1.shape, Test1.shape)

sigma = np.std(X1, axis = 0)
mean = np.mean(X1, axis = 0)

X1 = normalize(sigma, mean, X1)
Test1 = normalize(sigma , mean, Test1)
print(X1.shape, Test1.shape)


regr = linear_model.LogisticRegression(solver='lbfgs',max_iter=1000, C=0.1)
regr.fit(X1, Y)
score1 = regr.score(X1, Y)
print(score1)
Prediction = regr.predict(Test1)
image_id = np.arange(1,Prediction.shape[0]+1)
pd.DataFrame({"ImageId": image_id, "Label": Prediction}).to_csv('out_reg1.csv', 
                                                                      index=False, header=True)

#### I submitted the data and got an accuracy of 0.9815. Now we will try to train data with SVM and kernel = 'rbf'. Note we will not use polynomial features with SVM since SVM maps the data in higher dimensions so there is no point in including the polynomial features, also SVM doesn't perform well with too many features.

##### 2.SVM

In [0]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC

def supportVM(X_train2, y_train2, X_test2, y_test2, penalty):
    regr = SVC(kernel ='rbf', C=penalty)
    regr.fit(X_train2, y_train2)
    score1 = regr.score(X_train2, y_train2)
    score2 = regr.score(X_test2, y_test2)
    print(score1, score2)
    Prediction = regr.predict(X_test2)
    return score1, score2, Prediction

X_train1, X_test1 = PrincipalComponents(0.81)
print(X_test1.shape, X_train1.shape)

sigma = np.std(X_train1, axis = 0)
mean = np.mean(X_train1, axis = 0)

X_train1 = normalize(sigma , mean , X_train1)
X_test1 = normalize(sigma , mean, X_test1)

J1 = []
J2 = []
for i in range(25000, X_train1.shape[0], 1500):
    score1, score2, Prediction = supportVM(X_train1[:i+1,:], 
                                                    y_train[:i+1, ], X_test1[:i+1,:], y_test[:i+1],10)
    J1.append(1-score1)
    J2.append(1-score2)
    
plt.plot(J2, 'b-', label = "CV error")
plt.plot(J1, 'r-', label ="training error")
plt.legend()
plt.ylabel("Error")
plt.xlabel("No of Iterations")
plt.show()

- So it seems that this model is performin a little bit better, So we will go with it, Let's save the prediction and save it.

##### Saving prediction trained with SVM

In [0]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC
# poly = PolynomialFeatures(include_bias = False)

X1 = X.copy()
Test1 = Test.copy()

pca = PCA(n_components= 0.8388)
X1 = pca.fit_transform(X1)
Test1 = pca.transform(Test)

print(X1.shape, Test1.shape)

print(X1.shape, Test1.shape)

sigma = np.std(X1, axis = 0)
mean = np.mean(X1, axis = 0)

X1 = normalize(sigma, mean, X1)
Test1 = normalize(sigma , mean, Test1)
print(X1.shape, Test1.shape)


regr = SVC(kernel ='rbf', C=10)
regr.fit(X1, Y)
score1 = regr.score(X1, Y)
print(score1)
Prediction = regr.predict(Test1)
Prediction = regr.predict(Test1)
image_id = np.arange(1,Prediction.shape[0]+1)
pd.DataFrame({"ImageId": image_id, "Label": Prediction}).to_csv('out_svm.csv', 
                                                                      index=False, header=True)

** I got an accuracy of 0.98442 on leaderboad with above model**.
I am learning CNN, So in future I will use that to train data and most probably I will launch a kernel also .Please give your valuable feedback.